In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
data_path = '../sensor_data/'
users = ['s1621503',
 's1746788',
 's1701688',
 's1710228',
 's1721039',
 's1616573',
 's1758009',
 's1660711']
full_paths = [data_path + usr + '/' + filename for usr in users for filename in os.listdir(data_path + usr)]
header_size = 5

In [3]:
# class to store recording data - and info about location and activity type
class DataRecording:  
    def __init__(self, sensor_position, sensor_side, activity, activity_id):
        self.sensor_position = sensor_position
        self.sensor_side = sensor_side
        self.activity = activity
        self.activity_id = activity_id
    
    @classmethod
    def create_recording(cls, filepath):
        with open(filepath) as f:
            head = [next(f).rstrip().split('# ')[1].split(':')[1].strip() for x in range(header_size)]
            recording = DataRecording(head[0], head[1], head[2], head[3])
        df = pd.read_csv(filepath, header=5)
        recording.data = df[['accel_x', 'accel_y', 'accel_z']]
        return recording

In [4]:
# read all files into a DataRecording 
all_data = [] # all DataRecordings
for path in full_paths:
    try:
        all_data.append(DataRecording.create_recording(path))
    except:
        pass
        print(path)

../sensor_data/s1721039/.DS_Store
../sensor_data/s1721039/list.txt
../sensor_data/s1721039/PDIoT Data Collection Protocol.docx
../sensor_data/s1721039/my_copy.sh


In [5]:
# Activities selected for classification
allowed_activites = ['Climbing stairs', 'Descending stairs', 'Walking at normal speed', 'Running', 'Standing']
def get_activity_id(activity):
    return allowed_activites.index(activity)

In [6]:
# Select only allowed activities from right front pocket
right_pocket_data = [x for x in all_data if 
                        x.sensor_position == 'Front pocket (trouser)' 
                            and 
                        x.sensor_side == 'Right'
                           and
                        x.activity in allowed_activites
                       ]
right_pocket_data[0].data

,accel_x,accel_y,accel_z
0,-0.347168,-0.996155,0.004822
1,-0.339600,-0.986877,0.061218
2,-0.216797,-1.018127,0.129089
3,-0.149902,-0.910950,-0.038879
4,-0.267090,-0.824036,0.064636
...,...,...,...
283,-0.219727,-0.969543,-0.039368
284,-0.219971,-0.976868,-0.067932
285,-0.140869,-0.978088,-0.158997
286,-0.122314,-1.065491,-0.265930


In [7]:
# function to return all overlapping windows from a dataframe
# input a 2D nxm dataframe
# output y x win_size x m - where y in the number of windows
def get_overlapping_windows(df, win_size, overlap):
    rows  = df.shape[0]
    assert(overlap < 1)
    assert(overlap >= 0)
    assert(win_size <= rows)

    increment = int(win_size * (1 - overlap))
    result = []
    for i in range(0, rows - win_size, increment):
        feature_vector = df[i:i+win_size]
        result.append(feature_vector)
    return np.array(result)

In [8]:
# create a dataset
# each row is a window of window x 3 of x, y, z acceleration data
# y corresponds to activity_id
def create_dataset(data_objects, win_size, overlap):
    X = None
    y = None
    for d_o in data_objects:
        windows = get_overlapping_windows(d_o.data, win_size, overlap)
        
        # create DataSet
        if X is None:
            X = windows
        else:
            X = np.vstack((X, windows))
        
        # Create Labels
        if y is None:
            y = np.repeat(get_activity_id(d_o.activity), windows.shape[0])
        else:
            ys = np.repeat(get_activity_id(d_o.activity), windows.shape[0])
            y = np.concatenate((y, ys))
 
    
    return X, y
        
        


In [9]:
# Create the dataset
X, y = create_dataset(right_pocket_data, 50, 0.5)
y = tf.keras.utils.to_categorical(y, num_classes=5)


In [10]:
n_timesteps, n_features, n_outputs = X.shape[1], X.shape[2], y.shape[1]

In [11]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = tf.keras.Sequential()
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [12]:
# create test and train sets
indices = np.random.permutation(X.shape[0])
boundary = int(X.shape[0] * 0.8)
train_idx, test_idx = indices[:boundary], indices[boundary:]
Xtrain, Ytrain, Xtest, Ytest = X[train_idx,:], y[train_idx,:], X[test_idx,:], y[test_idx,:]

In [1]:
score = evaluate_model(Xtrain, Ytrain, Xtest, Ytest)

NameError: name 'evaluate_model' is not defined

In [3]:
# save model as tflite
def save_tflite_model(model):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    with open('model.tflite', 'wb') as f:
        f.write(tflite_model)